# BNY Vendor Contract QA Agent Documentation POC

## Pre-requisites

In [1]:
%pip install duckduckgo-search beautifulsoup4 --quiet

Note: you may need to restart the kernel to use updated packages.


In [2]:
# load openai api key
import os

from dotenv import load_dotenv
load_dotenv()

if not 'OPENAI_API_KEY' in os.environ:
    raise ValueError('OPENAI_API_KEY is not set')

GPT_MODEL = "gpt-3.5-turbo"
os.environ["OPENAI_GPT_MODEL"] = GPT_MODEL

In [3]:
import sys

sys.path.append(os.getcwd())

from utils import (
    client,
    init_db,
    get_schema_description,
    get_tools_spec,
    show_json,
)

In [4]:
init_db()

# Create Agent

### Load Tools and DB Schema Spec

In [5]:
tools_spec = get_tools_spec()

Loaded 2 tool definitions
[
      {
            "type": "function",
            "function": {
                  "name": "query_database",
                  "description": "Query the contract database to retrieve information about contracts with a particular vendor",
                  "parameters": {
                        "type": "object",
                        "properties": {
                              "query": {
                                    "type": "string",
                                    "description": "SQL query to execute against the database"
                              }
                        },
                        "required": [
                              "query"
                        ]
                  }
            }
      },
      {
            "type": "function",
            "function": {
                  "name": "search_online",
                  "description": "Search the web for information",
                  "parameters": {
             

In [6]:
print(get_schema_description())

Table: vendors
  vendor_id: TEXT
  vendor_name: TEXT
  category: TEXT
  total_spend: INTEGER
  contracts: TEXT
Table: contracts
  contract_id: TEXT
  vendor_id: TEXT
  start_date: TEXT
  end_date: TEXT
  contract_value: INTEGER
  terms_conditions: TEXT
  products_description: TEXT



### Create an Assistant

In [7]:
AGENT_SYSTEM_PROMPT = f"""
# Mission:
You are an AI Agent that helps employees answer questions they might have about everything related to software vendors.
You will be asked questions such as "Do we have a vendor for cloud storage?" or "I need a tool for project management".
You should use the tools available to you as well as semantic search on the documents you have access to to answer these questions.

# Guidelines:
For "Do we have a vendor for cloud storage?", you could use the `query_database` to query the contracts database for cloud storage vendors.
Then you could search your document repository for information on the vendors you found.
If none are found, then you might search online using the `search_online` tool to discover new vendors.
Or, for the question "I need a tool for project management", if you cannot find a relevant vendor in the database,
  you could use the `search_online` tool to find out if any existing vendors provide project management tools.

# Constraints:
You should always try and find relevant information from the database.
You can search online to find new information or confirm information that you already know.
You should only fall back to your existing knowledge of vendors to help you come up with good search queries or when you want to enrich your answers.
  - For example, if the user is asking for a certain product and you find a vendor in the db that doesn't mention that product but you know they offer it, you can share that with the user.
You should only use your document retrieval system to find extra information related to vendors found in the database or online - essentially to enrich your knowledge before answering.
Don't make assumptions about what values to plug into functions. Ask for clarification if a user request is ambiguous.

# DB Schema:
{get_schema_description()}
""".strip()


# Create an agent using the OpenAI Assistants API
# agent = client.beta.assistants.create(
#     name="Vendor Contract Q/A Assistant",
#     instructions=AGENT_SYSTEM_PROMPT,
#     model="gpt-4-turbo",
#     tools=tools_spec,
# )
# show_json(agent)
# AGENT_ID = agent.id

# Or use an existing agent
AGENT_ID = "asst_sAavPqXvzJNzFtoUmtfF0qCj"

# Use the Agent

In [8]:
from utils import AgentEventHandler

os.environ["DEBUG"] = "0"

thread = client.beta.threads.create()


def send_message(user_message):
    message = client.beta.threads.messages.create(
        thread_id=thread.id,
        role="user",
        content=user_message,
    )
    
    with client.beta.threads.runs.stream(
        thread_id=thread.id,
        assistant_id=AGENT_ID,
        event_handler=AgentEventHandler()
    ) as stream:
        stream.until_done()

In [9]:
send_message("Can you find me a vendor that might offer a serverless mysql db?")

> Querying database with:  {'query': "SELECT vendor_name FROM vendors WHERE category = 'Database Services' OR category = 'Cloud Computing'"}
> Result:  [["Amazon Web Services"]]


> Searching online with:  {'search_type': 'browse', 'query': 'Amazon Web Services serverless MySQL'}
> Result:  Error calling tool: search_online: https://links.duckduckgo.com/d.js?q=Amazon+Web+Services+serverless+MySQL&kl=wt-wt&l=wt-wt&p=&s=0&df=&vqd=4-37991713874655709774203304645897858903&ex=-1 202 Ratelimit


> Receiving message from agent:


HTML(value='\n<div id="message_container">\n\n</div>\n<style>\n#message_container {\n    padding: 10px;\n    b…

In [10]:
send_message("Are you sure?")

> Querying database with:  {'query': "SELECT vendors.vendor_name, contracts.products_description FROM vendors JOIN contracts ON vendors.vendor_id = contracts.vendor_id WHERE vendors.vendor_name = 'Amazon Web Services' AND (contracts.products_description LIKE '%serverless%' AND contracts.products_description LIKE '%MySQL%')"}
> Result:  []


> Receiving message from agent:


HTML(value='\n<div id="message_container">\n\n</div>\n<style>\n#message_container {\n    padding: 10px;\n    b…

In [11]:
send_message("Do we have a contract or vendor relationship with AWS?")

> Querying database with:  {'query': "SELECT vendor_name, contracts FROM vendors WHERE vendor_name = 'Amazon Web Services'"}
> Result:  [["Amazon Web Services", "C007,C008"]]


> Receiving message from agent:


HTML(value='\n<div id="message_container">\n\n</div>\n<style>\n#message_container {\n    padding: 10px;\n    b…

In [12]:
send_message("Yeah can you give me a full summary of our relationship/contracts with them?")

> Querying database with:  {'query': "SELECT contracts.contract_id, contracts.vendor_id, contracts.start_date, contracts.end_date, contracts.contract_value, contracts.terms_conditions, contracts.products_description FROM contracts JOIN vendors ON contracts.vendor_id = vendors.vendor_id WHERE vendors.vendor_name = 'Amazon Web Services'"}
> Result:  [["C007", "V005", "2022-08-01", "2025-07-31", 120000, "Cloud services provisioning - unlimited consulting hours to setup and provision new AWS services for 3 years - renewal option available.", "Amazon Web Services (AWS) cloud services provision."], ["C008", "V005", "2021-09-01", "2024-08-31", 95000, "Pay-as-you-go AWS cloud services.", "Amazon Web Services (AWS) cloud services"]]


> Receiving message from agent:


HTML(value='\n<div id="message_container">\n\n</div>\n<style>\n#message_container {\n    padding: 10px;\n    b…